In [1]:
pip install google-api-python-client google-auth google-auth-oauthlib google-auth-httplib2


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 11.6 MB 284 kB/s eta 0:00:01
     |████████████████████████████████| 189 kB 282 kB/s eta 0:00:01
  Using cached httplib2-0.22.0-py3-none-any.whl (96 kB)
     |████████████████████████████████| 139 kB 266 kB/s eta 0:00:01
  Using cached uritemplate-4.1.1-py2.py3-none-any.whl (10 kB)
  Using cached cachetools-5.3.3-py3-none-any.whl (9.3 kB)
  Using cached rsa-4.9-py3-none-any.whl (34 kB)
     |████████████████████████████████| 181 kB 211 kB/s eta 0:00:01
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
     |████████████████████████████████| 48 kB 222 kB/s eta 0:00:01
     |████████████████████████████████| 394 kB 181 kB/s eta 0:00:01
     |████████████████████████████████| 229 kB 240 kB/s eta 0:00:01
  Using cached pyparsing-3.1.2-py3-none-any.whl (103 kB)
     |████████████████████████████████| 85 kB 213 kB/s eta 0:00:01
  Using cached idna-3.7-py3-none-any.whl (

## XAV -- Each video like and comments 


In [11]:
import os
import google_auth_oauthlib.flow
import googleapiclient.discovery

# Set up OAuth credentials
os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"  # Only for development/testing
flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
    "xav_client.json",
    scopes=["https://www.googleapis.com/auth/youtube.readonly"]
)
credentials = flow.run_local_server(port=0)

# Create a YouTube Data API client
youtube = googleapiclient.discovery.build("youtube", "v3", credentials=credentials)

# Get the playlist ID of the channel's uploaded videos
channel_id = "UCaufk4I0YqJZ-yIk8TNMS9w"
response = youtube.channels().list(
    part="contentDetails",
    id=channel_id
).execute()

if "items" in response:
    playlist_id = response["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]

    # Retrieve the list of videos in the playlist
    videos = []
    next_page_token = None
    while True:
        playlist_items_response = youtube.playlistItems().list(
            part="contentDetails",
            playlistId=playlist_id,
            maxResults=50,
            pageToken=next_page_token
        ).execute()

        videos.extend(playlist_items_response["items"])
        next_page_token = playlist_items_response.get("nextPageToken")

        if not next_page_token:
            break

    # Retrieve analytics for each video
    for video in videos:
        video_id = video["contentDetails"]["videoId"]
        video_response = youtube.videos().list(
            part="snippet,statistics",
            id=video_id
        ).execute()

        if "items" in video_response:
            video_info = video_response["items"][0]
            video_title = video_info["snippet"]["title"]
            video_stats = video_info["statistics"]
            print(f"Video Title: {video_title}")
            print(f"Video ID: {video_id}")
            print(f"Views: {video_stats['viewCount']}")
            print(f"Likes: {video_stats['likeCount']}")
            print(f"Comments: {video_stats['commentCount']}")
            print("---")
        else:
            print(f"Video ID: {video_id}")
            print("Video not found.")
            print("---")
else:
    print("Channel not found.")


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=700561766577-d307303isgapjpl69e7lgtv2gitc665l.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A54672%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=QveOmnMWlZQdPROFRJMT4ChdH4vKEs&access_type=offline
Video Title: راهنمای دم‌آوری قهوه هندوراس با دم‌افزار اوریگامی | XAV Recipe 2
Video ID: Od0yb7FqVik
Views: 545
Likes: 27
Comments: 14
---
Video Title: درخت قهوه
Video ID: MBZsQD0jAlw
Views: 191
Likes: 7
Comments: 2
---
Video Title: Percolation Variables | متغييرهای دم آوری چكانه ای
Video ID: xeJqDpzwD_o
Views: 409
Likes: 20
Comments: 4
---
Video Title: What is specialty coffee? | قهوه تخصصی چيست؟
Video ID: IBA3Oxb2F2Y
Views: 448
Likes: 18
Comments: 4
---
Video Title: Roasting | برشته كاری
Video ID: DbBrZ1NSRog
Views: 1596
Likes: 39
Comments: 9
---
Video Title: Extraction | عصاره گيری
Video ID: S0v_iyJGZus
Views: 2086
Likes: 64

## Channle Data

In [1]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import csv

from tabulate import tabulate

SCOPES = ['https://www.googleapis.com/auth/yt-analytics.readonly']

API_SERVICE_NAME = 'youtubeAnalytics'
API_VERSION = 'v2'
CLIENT_SECRETS_FILE = 'xav_client.json'

def get_service():
  flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
  credentials = flow.run_local_server()
  # alternatively (https://github.com/onlyphantom/youtube_api_python/pull/3/files):
  # credentials = flow.run_local_server()
  return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

def execute_api_request(client_library_function, **kwargs):
  response = client_library_function(
    **kwargs
  ).execute()
  return response

def create_table(table, headers=None):
    if headers:
        headerstring = "\t{}\t" * len(headers)
        print(headerstring.format(*headers))

    rowstring = "\t{}\t" * len(table[0])

    for row in table:
        print(rowstring.format(*row))

def create_csv(table, headers=None, filename='output_channel.csv'):
    with open(filename, 'w') as f:
        writer = csv.writer(f)
        if headers:
            writer.writerow(headers)
        writer.writerows(table)

if __name__ == '__main__':

    youtubeAnalytics = get_service()
    result = execute_api_request(
        youtubeAnalytics.reports().query,
        ids='channel==UCaufk4I0YqJZ-yIk8TNMS9w',
        startDate='2021-10-06',
        endDate='2023-10-31',
        metrics= 'estimatedMinutesWatched,views,likes,comments,subscribersGained,shares,averageViewDuration,annotationImpressions,annotationClickThroughRate',
        dimensions= 'day',        
    )
    headers = ['date','estMinutesWatched', 'views', 'likes','comments', 'subscribersGained', 'shares', 'averageViewDuration','annotationImpressions','annotationClickThroughRate']
    # create_table(result['rows'], headers=headers)
    create_csv(result['rows'], headers=headers)
    print(tabulate(result['rows'], headers=headers, tablefmt="pretty"))

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=700561766577-d307303isgapjpl69e7lgtv2gitc665l.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyt-analytics.readonly&state=CkHVyTOv9QjP4rqzEAY5lFXUBzRYmd&access_type=offline
+------------+-------------------+-------+-------+----------+-------------------+--------+---------------------+-----------------------+----------------------------+
|    date    | estMinutesWatched | views | likes | comments | subscribersGained | shares | averageViewDuration | annotationImpressions | annotationClickThroughRate |
+------------+-------------------+-------+-------+----------+-------------------+--------+---------------------+-----------------------+----------------------------+
| 2021-10-08 |         8         |   3   |   2   |    0     |         1         |   0    |         175         |           0        

##  Each Video " How viowers find this video"Data

In [1]:
import os
import googleapiclient.discovery
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow

## achieving video ids

# Set up OAuth credentials
os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"  # Only for development/testing
flow = InstalledAppFlow.from_client_secrets_file(
    "xav_client.json",
    scopes=["https://www.googleapis.com/auth/youtube.readonly"]
)
credentials = flow.run_local_server(port=0)

# Create a YouTube Data API client
youtube = googleapiclient.discovery.build("youtube", "v3", credentials=credentials)

# Get the playlist ID of the channel's uploaded videos
channel_id = "UCaufk4I0YqJZ-yIk8TNMS9w"
response = youtube.channels().list(
    part="contentDetails",
    id=channel_id
).execute()

if "items" in response:
    playlist_id = response["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]

    # Retrieve the list of videos in the playlist
    videos = []
    next_page_token = None
    while True:
        playlist_items_response = youtube.playlistItems().list(
            part="contentDetails",
            playlistId=playlist_id,
            maxResults=50,
            pageToken=next_page_token
        ).execute()

        videos.extend(playlist_items_response["items"])
        next_page_token = playlist_items_response.get("nextPageToken")

        if not next_page_token:
            break

video_ids = []
for i in videos:
    video_id = i["contentDetails"]["videoId"]
    video_ids.append(video_id)




SCOPES = ['https://www.googleapis.com/auth/yt-analytics.readonly']

API_SERVICE_NAME = 'youtubeAnalytics'
API_VERSION = 'v2'
CLIENT_SECRETS_FILE = 'xav_client.json'

flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
credentials = flow.run_local_server()
# alternatively (https://github.com/onlyphantom/youtube_api_python/pull/3/files):
# credentials = flow.run_local_server()
youtube_analytics = build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

# Iterate over the video IDs
for video_id in video_ids:
    try:
        # Retrieve viewer sources
        analytics_response = youtube_analytics.reports().query(
            ids="channel==UCaufk4I0YqJZ-yIk8TNMS9w",
            startDate="2021-01-01",
            endDate="2023-10-31",
            metrics="views",
            dimensions="insightTrafficSourceType",
            filters=f"video=={video_id}"
        ).execute()

        # Extract viewer sources
        viewer_sources = analytics_response['rows']

        # Process the results
        print(f"Video ID: {video_id}")
        print("Viewer Sources:")
        for source in viewer_sources:
            print(f"- {source[0]}: {source[1]} views")
        print("------------------------")

    except HttpError as e:
        print(f"An error occurred: {e}")





    

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=700561766577-d307303isgapjpl69e7lgtv2gitc665l.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A58901%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=r2rMpFKIdDZhvEeB9hbFsDWvZymBMf&access_type=offline
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=700561766577-d307303isgapjpl69e7lgtv2gitc665l.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyt-analytics.readonly&state=BTfn7lJP496IhVFWwe2AUvfIvvJ2Xi&access_type=offline
Video ID: NX5tWv2twvs
Viewer Sources:
------------------------
Video ID: GPsLRwHzmZU
Viewer Sources:
------------------------
Video ID: Od0yb7FqVik
Viewer Sources:
- NO_LINK_OTHER: 59 views
- SUBSCRIBER: 89 views
- YT_CHANNEL: 114 views
- YT_SEARCH: 56 views
- REL

## Each video "Basic Stats" Data

In [7]:
import os
import googleapiclient.discovery
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow

## achieving video ids

# Set up OAuth credentials
os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"  # Only for development/testing
flow = InstalledAppFlow.from_client_secrets_file(
    "xav_client.json",
    scopes=["https://www.googleapis.com/auth/youtube.readonly"]
)
credentials = flow.run_local_server(port=0)

# Create a YouTube Data API client
youtube = googleapiclient.discovery.build("youtube", "v3", credentials=credentials)

# Get the playlist ID of the channel's uploaded videos
channel_id = "UCaufk4I0YqJZ-yIk8TNMS9w"
response = youtube.channels().list(
    part="contentDetails",
    id=channel_id
).execute()

if "items" in response:
    playlist_id = response["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]

    # Retrieve the list of videos in the playlist
    videos = []
    next_page_token = None
    while True:
        playlist_items_response = youtube.playlistItems().list(
            part="contentDetails",
            playlistId=playlist_id,
            maxResults=50,
            pageToken=next_page_token
        ).execute()

        videos.extend(playlist_items_response["items"])
        next_page_token = playlist_items_response.get("nextPageToken")

        if not next_page_token:
            break

video_ids = []
for i in videos:
    video_id = i["contentDetails"]["videoId"]
    video_ids.append(video_id)
    



from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import csv

from tabulate import tabulate

SCOPES = ['https://www.googleapis.com/auth/yt-analytics.readonly']

API_SERVICE_NAME = 'youtubeAnalytics'
API_VERSION = 'v2'
CLIENT_SECRETS_FILE = 'xav_client.json'

def get_service():
  flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
  credentials = flow.run_local_server()
  # alternatively (https://github.com/onlyphantom/youtube_api_python/pull/3/files):
  # credentials = flow.run_local_server()
  return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

def execute_api_request(client_library_function, **kwargs):
  response = client_library_function(
    **kwargs
  ).execute()
  return response

def create_table(table, headers=None):
    if headers:
        headerstring = "\t{}\t" * len(headers)
        print(headerstring.format(*headers))

    rowstring = "\t{}\t" * len(table[0])

    for row in table:
        print(rowstring.format(*row))

def create_csv(table, headers=None, filename=f'output{video_id}.csv'):
    with open(filename, 'w') as f:
        writer = csv.writer(f)
        if headers:
            writer.writerow(headers)
        writer.writerows(table)

if __name__ == '__main__':
    youtubeAnalytics = get_service()
    for i in video_ids:
        result = execute_api_request(
          youtubeAnalytics.reports().query,
          ids='channel==UCaufk4I0YqJZ-yIk8TNMS9w',
          startDate='2019-10-06',
          endDate='2023-10-31',
          metrics= 'estimatedMinutesWatched,views,likes,comments,subscribersGained,shares,videosAddedToPlaylists,averageViewPercentage,videosAddedToPlaylists',
          dimensions= None,
          filters= f"video=={i}"
          
      )
        headers = ['estMinutesWatched', 'views', 'likes','comments', 'subscribersGained', 'shares','videosAddedToPlaylists' ,'averageViewPercentage', 'videosAddedToPlaylists']
        # create_table(result['rows'], headers=headers)
        create_csv(result['rows'], headers=headers,filename= f'{i}.csv' )
        print(tabulate(result['rows'], headers=headers, tablefmt="pretty"))

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=700561766577-d307303isgapjpl69e7lgtv2gitc665l.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A55313%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=cacVxWRbjNYiJJeCmDAYLxmw7oRpHg&access_type=offline
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=700561766577-d307303isgapjpl69e7lgtv2gitc665l.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyt-analytics.readonly&state=Md33SioULCReqaPpPyYrv9j7k1gLlQ&access_type=offline
+-------------------+-------+-------+-------------------+--------+------------------------+-----------------------+------------------------+
| estMinutesWatched | views | likes | subscribersGained | shares | videosAddedToPlaylists | averageViewPercentage | videosA

## Time Based Data

In [2]:
import os
import googleapiclient.discovery
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow

## achieving video ids

# Set up OAuth credentials
os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"  # Only for development/testing
flow = InstalledAppFlow.from_client_secrets_file(
    "xav_client.json",
    scopes=["https://www.googleapis.com/auth/youtube.readonly"]
)
credentials = flow.run_local_server(port=0)

# Create a YouTube Data API client
youtube = googleapiclient.discovery.build("youtube", "v3", credentials=credentials)

# Get the playlist ID of the channel's uploaded videos
channel_id = "UCaufk4I0YqJZ-yIk8TNMS9w"
response = youtube.channels().list(
    part="contentDetails",
    id=channel_id
).execute()

if "items" in response:
    playlist_id = response["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]

    # Retrieve the list of videos in the playlist
    videos = []
    next_page_token = None
    while True:
        playlist_items_response = youtube.playlistItems().list(
            part="contentDetails",
            playlistId=playlist_id,
            maxResults=50,
            pageToken=next_page_token
        ).execute()

        videos.extend(playlist_items_response["items"])
        next_page_token = playlist_items_response.get("nextPageToken")

        if not next_page_token:
            break

video_ids = []
for i in videos:
    video_id = i["contentDetails"]["videoId"]
    video_ids.append(video_id)
    



from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import csv
import matplotlib.pyplot as plt
from tabulate import tabulate

SCOPES = ['https://www.googleapis.com/auth/yt-analytics.readonly']

API_SERVICE_NAME = 'youtubeAnalytics'
API_VERSION = 'v2'
CLIENT_SECRETS_FILE = 'xav_client.json'

def get_service():
  flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
  credentials = flow.run_local_server()
  # alternatively (https://github.com/onlyphantom/youtube_api_python/pull/3/files):
  # credentials = flow.run_local_server()
  return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

def execute_api_request(client_library_function, **kwargs):
  response = client_library_function(
    **kwargs
  ).execute()
  return response

def create_table(table, headers=None):
    if headers:
        headerstring = "\t{}\t" * len(headers)
        print(headerstring.format(*headers))

    rowstring = "\t{}\t" * len(table[0])

    for row in table:
        print(rowstring.format(*row))

def create_csv(table, headers=None, filename=f'output{video_id}.csv'):
    with open(filename, 'w') as f:
        writer = csv.writer(f)
        if headers:
            writer.writerow(headers)
        writer.writerows(table)

if __name__ == '__main__':
    youtubeAnalytics = get_service()
    for i in video_ids:
        result = execute_api_request(
          youtubeAnalytics.reports().query,
          ids='channel==UCaufk4I0YqJZ-yIk8TNMS9w',
          startDate='2023-11-06',
          endDate='2023-11-27',
          metrics= 'estimatedMinutesWatched,views,likes,comments,subscribersGained,shares,videosAddedToPlaylists,averageViewPercentage',
          dimensions= 'day',
          filters= f"video=={i}"
          
       )


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=700561766577-d307303isgapjpl69e7lgtv2gitc665l.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A49838%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=m5uGKy2cqmKyK76xYIC4sAemc8yaSD&access_type=offline
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=700561766577-d307303isgapjpl69e7lgtv2gitc665l.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyt-analytics.readonly&state=B8W4YISZOG4n1eWueqL9qWvOb7pWp5&access_type=offline
+------------+-------------------+-------+-------+----------+-------------------+--------+------------------------+-----------------------+
|            | estMinutesWatched | views | likes | comments | subscribersGained | shares | videosAddedToPlaylists | averageV

In [17]:
import matplotlib.pyplot as plt
# date = []
# views = []
# estimated_min =[]
# for i in range(len(result["rows"])):
#     date.append(result["rows"][i][0][-2] + result["rows"][i][0][-1])
#     views.append(result["rows"][i][1])
#     estimated_min.append(result["rows"][i][2])

# # Create a figure and a set of subplots
# fig, axs = plt.subplots(2, figsize= (16,8))  # 2 rows of subplots

# # Plot data on the first subplot
# axs[0].plot(date, views)
# axs[0].set_title('views')

# # Plot data on the second subplot
# axs[1].plot(date, estimated_min)
# axs[1].set_title('\n \n Estimated minutes watch', fontsize= 12)

# plt.show()

result["rows"][0][2]

['2023-11-07', 2, 1, 0, 0, 0, 0, 1, 32.08]

## Geography Data

In [2]:
import os
import googleapiclient.discovery
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow

## achieving video ids

# Set up OAuth credentials
os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"  # Only for development/testing
flow = InstalledAppFlow.from_client_secrets_file(
    "xav_client.json",
    scopes=["https://www.googleapis.com/auth/youtube.readonly"]
)
credentials = flow.run_local_server(port=0)

# Create a YouTube Data API client
youtube = googleapiclient.discovery.build("youtube", "v3", credentials=credentials)

# Get the playlist ID of the channel's uploaded videos
channel_id = "UCaufk4I0YqJZ-yIk8TNMS9w"
response = youtube.channels().list(
    part="contentDetails",
    id=channel_id
).execute()

if "items" in response:
    playlist_id = response["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]

    # Retrieve the list of videos in the playlist
    videos = []
    next_page_token = None
    while True:
        playlist_items_response = youtube.playlistItems().list(
            part="contentDetails",
            playlistId=playlist_id,
            maxResults=50,
            pageToken=next_page_token
        ).execute()

        videos.extend(playlist_items_response["items"])
        next_page_token = playlist_items_response.get("nextPageToken")

        if not next_page_token:
            break

video_ids = []
for i in videos:
    video_id = i["contentDetails"]["videoId"]
    video_ids.append(video_id)
    



from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import csv

from tabulate import tabulate

SCOPES = ['https://www.googleapis.com/auth/yt-analytics.readonly']

API_SERVICE_NAME = 'youtubeAnalytics'
API_VERSION = 'v2'
CLIENT_SECRETS_FILE = 'xav_client.json'

def get_service():
  flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
  credentials = flow.run_local_server()
  # alternatively (https://github.com/onlyphantom/youtube_api_python/pull/3/files):
  # credentials = flow.run_local_server()
  return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

def execute_api_request(client_library_function, **kwargs):
  response = client_library_function(
    **kwargs
  ).execute()
  return response

def create_table(table, headers=None):
    if headers:
        headerstring = "\t{}\t" * len(headers)
        print(headerstring.format(*headers))

    rowstring = "\t{}\t" * len(table[0])

    for row in table:
        print(rowstring.format(*row))

def create_csv(table, headers=None, filename=f'output{video_id}.csv'):
    with open(filename, 'w') as f:
        writer = csv.writer(f)
        if headers:
            writer.writerow(headers)
        writer.writerows(table)

if __name__ == '__main__':
    youtubeAnalytics = get_service()
    for i in video_ids:
        result = execute_api_request(
          youtubeAnalytics.reports().query,
          ids='channel==UCaufk4I0YqJZ-yIk8TNMS9w',
          startDate='2019-10-06',
          endDate='2023-10-31',
          metrics= 'estimatedMinutesWatched,views,likes,comments,subscribersGained,shares,videosAddedToPlaylists,averageViewPercentage',
          dimensions= 'country',
          filters= f"video=={i}"
          
      )
        headers = ['estMinutesWatched', 'views', 'likes','comments', 'subscribersGained', 'shares','videosAddedToPlaylists' ,'averageViewPercentage']
        # create_table(result['rows'], headers=headers)
        create_csv(result['rows'], headers=headers,filename= f'{i}.csv' )
        print(tabulate(result['rows'], headers=headers, tablefmt="pretty"))

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=700561766577-d307303isgapjpl69e7lgtv2gitc665l.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A58486%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=JU8JP3AFR452k7URJicXgv8X5RpB4J&access_type=offline
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=700561766577-d307303isgapjpl69e7lgtv2gitc665l.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyt-analytics.readonly&state=rRjlg6CeD3nwAf6nJcu5QyiYIQlt9m&access_type=offline
+-------------------+-------+-------+----------+-------------------+--------+------------------------+-----------------------+
| estMinutesWatched | views | likes | comments | subscribersGained | shares | videosAddedToPlaylists | averageViewPercentage |
+---------

## Data By Subscribe Status

In [1]:
import os
import googleapiclient.discovery
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow

## achieving video ids

# Set up OAuth credentials
os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"  # Only for development/testing
flow = InstalledAppFlow.from_client_secrets_file(
    "xav_client.json",
    scopes=["https://www.googleapis.com/auth/youtube.readonly"]
)
credentials = flow.run_local_server(port=0)

# Create a YouTube Data API client
youtube = googleapiclient.discovery.build("youtube", "v3", credentials=credentials)

# Get the playlist ID of the channel's uploaded videos
channel_id = "UCaufk4I0YqJZ-yIk8TNMS9w"
response = youtube.channels().list(
    part="contentDetails",
    id=channel_id
).execute()

if "items" in response:
    playlist_id = response["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]

    # Retrieve the list of videos in the playlist
    videos = []
    next_page_token = None
    while True:
        playlist_items_response = youtube.playlistItems().list(
            part="contentDetails",
            playlistId=playlist_id,
            maxResults=50,
            pageToken=next_page_token
        ).execute()

        videos.extend(playlist_items_response["items"])
        next_page_token = playlist_items_response.get("nextPageToken")

        if not next_page_token:
            break

video_ids = []
for i in videos:
    video_id = i["contentDetails"]["videoId"]
    video_ids.append(video_id)
    



from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import csv

from tabulate import tabulate

SCOPES = ['https://www.googleapis.com/auth/yt-analytics.readonly']

API_SERVICE_NAME = 'youtubeAnalytics'
API_VERSION = 'v2'
CLIENT_SECRETS_FILE = 'xav_client.json'

def get_service():
  flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
  credentials = flow.run_local_server()
  # alternatively (https://github.com/onlyphantom/youtube_api_python/pull/3/files):
  # credentials = flow.run_local_server()
  return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

def execute_api_request(client_library_function, **kwargs):
  response = client_library_function(
    **kwargs
  ).execute()
  return response

def create_table(table, headers=None):
    if headers:
        headerstring = "\t{}\t" * len(headers)
        print(headerstring.format(*headers))

    rowstring = "\t{}\t" * len(table[0])

    for row in table:
        print(rowstring.format(*row))

def create_csv(table, headers=None, filename=f'output{video_id}.csv'):
    with open(filename, 'w') as f:
        writer = csv.writer(f)
        if headers:
            writer.writerow(headers)
        writer.writerows(table)

if __name__ == '__main__':
    youtubeAnalytics = get_service()
    for i in video_ids:
        result = execute_api_request(
          youtubeAnalytics.reports().query,
          ids='channel==UCaufk4I0YqJZ-yIk8TNMS9w',
          startDate='2019-10-06',
          endDate='2023-10-31',
          metrics= 'estimatedMinutesWatched,views,likes,shares,videosAddedToPlaylists',
          dimensions= 'subscribedStatus,day',
          filters= f"video=={i}"
          
      )
        headers = ['estMinutesWatched', 'views', 'likes', 'shares','videosAddedToPlaylists']
        # create_table(result['rows'], headers=headers)
        create_csv(result['rows'], headers=headers,filename= f'{i}.csv' )
        print(tabulate(result['rows'], headers=headers, tablefmt="pretty"))

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=700561766577-d307303isgapjpl69e7lgtv2gitc665l.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A59685%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=WJ4DGhsTd5gfXKTaDjWJfMLxYjHEQ0&access_type=offline
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=700561766577-d307303isgapjpl69e7lgtv2gitc665l.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyt-analytics.readonly&state=xvYM2XU8QRsifUV96A32TGm1pBDhT9&access_type=offline
+-------------------+-------+-------+--------+------------------------+-----------------------+
| estMinutesWatched | views | likes | shares | videosAddedToPlaylists | averageViewPercentage |
+-------------------+-------+-------+--------+------------------------+-

## Playback Locations

In [5]:
import os
import googleapiclient.discovery
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow

## achieving video ids

# Set up OAuth credentials
os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"  # Only for development/testing
flow = InstalledAppFlow.from_client_secrets_file(
    "xav_client.json",
    scopes=["https://www.googleapis.com/auth/youtube.readonly"]
)
credentials = flow.run_local_server(port=0)

# Create a YouTube Data API client
youtube = googleapiclient.discovery.build("youtube", "v3", credentials=credentials)

# Get the playlist ID of the channel's uploaded videos
channel_id = "UCaufk4I0YqJZ-yIk8TNMS9w"
response = youtube.channels().list(
    part="contentDetails",
    id=channel_id
).execute()

if "items" in response:
    playlist_id = response["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]

    # Retrieve the list of videos in the playlist
    videos = []
    next_page_token = None
    while True:
        playlist_items_response = youtube.playlistItems().list(
            part="contentDetails",
            playlistId=playlist_id,
            maxResults=50,
            pageToken=next_page_token
        ).execute()

        videos.extend(playlist_items_response["items"])
        next_page_token = playlist_items_response.get("nextPageToken")

        if not next_page_token:
            break

video_ids = []
for i in videos:
    video_id = i["contentDetails"]["videoId"]
    video_ids.append(video_id)
    



from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import csv

from tabulate import tabulate

SCOPES = ['https://www.googleapis.com/auth/yt-analytics.readonly']

API_SERVICE_NAME = 'youtubeAnalytics'
API_VERSION = 'v2'
CLIENT_SECRETS_FILE = 'xav_client.json'

def get_service():
  flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
  credentials = flow.run_local_server()
  # alternatively (https://github.com/onlyphantom/youtube_api_python/pull/3/files):
  # credentials = flow.run_local_server()
  return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

def execute_api_request(client_library_function, **kwargs):
  response = client_library_function(
    **kwargs
  ).execute()
  return response

def create_table(table, headers=None):
    if headers:
        headerstring = "\t{}\t" * len(headers)
        print(headerstring.format(*headers))

    rowstring = "\t{}\t" * len(table[0])

    for row in table:
        print(rowstring.format(*row))

def create_csv(table, headers=None, filename=f'output{video_id}.csv'):
    with open(filename, 'w') as f:
        writer = csv.writer(f)
        if headers:
            writer.writerow(headers)
        writer.writerows(table)

if __name__ == '__main__':
    youtubeAnalytics = get_service()
    for i in video_ids:
        result = execute_api_request(
          youtubeAnalytics.reports().query,
          ids='channel==UCaufk4I0YqJZ-yIk8TNMS9w',
          startDate='2019-10-06',
          endDate='2023-10-31',
          metrics= 'estimatedMinutesWatched,views',
          dimensions= 'insightPlaybackLocationType',
          filters= f"video=={i}"
          
      )
        headers = ['estMinutesWatched', 'views']
        # create_table(result['rows'], headers=headers)
        create_csv(result['rows'], headers=headers,filename= f'{i}.csv' )
        print(tabulate(result['rows'], headers=headers, tablefmt="pretty"))

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=700561766577-d307303isgapjpl69e7lgtv2gitc665l.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A58607%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=TrIsFAPaKXxVpdE4G7r7AUH0qxdqnJ&access_type=offline
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=700561766577-d307303isgapjpl69e7lgtv2gitc665l.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyt-analytics.readonly&state=Vldrz1RpxZNT97eACW1nZVqIcejrhp&access_type=offline
+-------------------+-------+
| estMinutesWatched | views |
+-------------------+-------+
+-------------------+-------+
+--------+-------------------+-------+
|        | estMinutesWatched | views |
+--------+-------------------+-------+
| WATCH  |       1695      

## Device type Data

In [1]:
import os
import googleapiclient.discovery
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow

## achieving video ids

# Set up OAuth credentials
os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"  # Only for development/testing
flow = InstalledAppFlow.from_client_secrets_file(
    "xav_client.json",
    scopes=["https://www.googleapis.com/auth/youtube.readonly"]
)
credentials = flow.run_local_server(port=0)

# Create a YouTube Data API client
youtube = googleapiclient.discovery.build("youtube", "v3", credentials=credentials)

# Get the playlist ID of the channel's uploaded videos
channel_id = "UCaufk4I0YqJZ-yIk8TNMS9w"
response = youtube.channels().list(
    part="contentDetails",
    id=channel_id
).execute()

if "items" in response:
    playlist_id = response["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]

    # Retrieve the list of videos in the playlist
    videos = []
    next_page_token = None
    while True:
        playlist_items_response = youtube.playlistItems().list(
            part="contentDetails",
            playlistId=playlist_id,
            maxResults=50,
            pageToken=next_page_token
        ).execute()

        videos.extend(playlist_items_response["items"])
        next_page_token = playlist_items_response.get("nextPageToken")

        if not next_page_token:
            break

video_ids = []
for i in videos:
    video_id = i["contentDetails"]["videoId"]
    video_ids.append(video_id)
    



from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import csv

from tabulate import tabulate

SCOPES = ['https://www.googleapis.com/auth/yt-analytics.readonly']

API_SERVICE_NAME = 'youtubeAnalytics'
API_VERSION = 'v2'
CLIENT_SECRETS_FILE = 'xav_client.json'

def get_service():
  flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
  credentials = flow.run_local_server()
  # alternatively (https://github.com/onlyphantom/youtube_api_python/pull/3/files):
  # credentials = flow.run_local_server()
  return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

def execute_api_request(client_library_function, **kwargs):
  response = client_library_function(
    **kwargs
  ).execute()
  return response

def create_table(table, headers=None):
    if headers:
        headerstring = "\t{}\t" * len(headers)
        print(headerstring.format(*headers))

    rowstring = "\t{}\t" * len(table[0])

    for row in table:
        print(rowstring.format(*row))

def create_csv(table, headers=None, filename=f'output{video_id}.csv'):
    with open(filename, 'w') as f:
        writer = csv.writer(f)
        if headers:
            writer.writerow(headers)
        writer.writerows(table)

if __name__ == '__main__':
    youtubeAnalytics = get_service()
    for i in video_ids:
        result = execute_api_request(
          youtubeAnalytics.reports().query,
          ids='channel==UCaufk4I0YqJZ-yIk8TNMS9w',
          startDate='2019-10-06',
          endDate='2023-10-31',
          metrics= 'estimatedMinutesWatched,views',
          dimensions= 'operatingSystem,deviceType',
          filters= f"video=={i}"
          
      )
        headers = ['estMinutesWatched', 'views']
        # create_table(result['rows'], headers=headers)
        create_csv(result['rows'], headers=headers,filename= f'{i}.csv' )
        print(tabulate(result['rows'], headers=headers, tablefmt="pretty"))

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=700561766577-d307303isgapjpl69e7lgtv2gitc665l.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A59943%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=41zPFMyxBaT0IeaaF5DrpvPulZEYzb&access_type=offline
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=700561766577-d307303isgapjpl69e7lgtv2gitc665l.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyt-analytics.readonly&state=4HEkyOztg9o9sFMvPAx4XBTiWYJWn4&access_type=offline
+-------------------+-------+
| estMinutesWatched | views |
+-------------------+-------+
+-------------------+-------+
+-----------+---------+-------------------+-------+
|           |         | estMinutesWatched | views |
+-----------+---------+-----------------

## Viewer demographics Data

In [3]:
import os
import googleapiclient.discovery
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow

## achieving video ids

# Set up OAuth credentials
os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"  # Only for development/testing
flow = InstalledAppFlow.from_client_secrets_file(
    "xav_client.json",
    scopes=["https://www.googleapis.com/auth/youtube.readonly"]
)
credentials = flow.run_local_server(port=0)

# Create a YouTube Data API client
youtube = googleapiclient.discovery.build("youtube", "v3", credentials=credentials)

# Get the playlist ID of the channel's uploaded videos
channel_id = "UCaufk4I0YqJZ-yIk8TNMS9w"
response = youtube.channels().list(
    part="contentDetails",
    id=channel_id
).execute()

if "items" in response:
    playlist_id = response["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]

    # Retrieve the list of videos in the playlist
    videos = []
    next_page_token = None
    while True:
        playlist_items_response = youtube.playlistItems().list(
            part="contentDetails",
            playlistId=playlist_id,
            maxResults=50,
            pageToken=next_page_token
        ).execute()

        videos.extend(playlist_items_response["items"])
        next_page_token = playlist_items_response.get("nextPageToken")

        if not next_page_token:
            break

video_ids = []
for i in videos:
    video_id = i["contentDetails"]["videoId"]
    video_ids.append(video_id)
    



from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import csv

from tabulate import tabulate

SCOPES = ['https://www.googleapis.com/auth/yt-analytics.readonly']

API_SERVICE_NAME = 'youtubeAnalytics'
API_VERSION = 'v2'
CLIENT_SECRETS_FILE = 'xav_client.json'

def get_service():
  flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
  credentials = flow.run_local_server()
  # alternatively (https://github.com/onlyphantom/youtube_api_python/pull/3/files):
  # credentials = flow.run_local_server()
  return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

def execute_api_request(client_library_function, **kwargs):
  response = client_library_function(
    **kwargs
  ).execute()
  return response

def create_table(table, headers=None):
    if headers:
        headerstring = "\t{}\t" * len(headers)
        print(headerstring.format(*headers))

    rowstring = "\t{}\t" * len(table[0])

    for row in table:
        print(rowstring.format(*row))

def create_csv(table, headers=None, filename=f'output{video_id}.csv'):
    with open(filename, 'w') as f:
        writer = csv.writer(f)
        if headers:
            writer.writerow(headers)
        writer.writerows(table)

if __name__ == '__main__':
    youtubeAnalytics = get_service()
    for i in video_ids:
        result = execute_api_request(
          youtubeAnalytics.reports().query,
          ids='channel==UCaufk4I0YqJZ-yIk8TNMS9w',
          startDate='2019-10-06',
          endDate='2023-10-31',
          metrics= 'viewerPercentage',
          dimensions= 'ageGroup,gender',
          filters= f"video=={i}"
          
      )
        headers = ['ageGroup', 'gender']
        # create_table(result['rows'], headers=headers)
        create_csv(result['rows'], headers=headers,filename= f'{i}.csv' )
        print(tabulate(result['rows'], headers=headers, tablefmt="pretty"))

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=700561766577-d307303isgapjpl69e7lgtv2gitc665l.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A60037%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=f2uj69yF5ikC33zhyMDT0rEwXi9xFn&access_type=offline
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=700561766577-d307303isgapjpl69e7lgtv2gitc665l.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyt-analytics.readonly&state=Q2lnFfRuHZay4Bs8CfW8Vc30OTl7zr&access_type=offline
+----------+--------+
| ageGroup | gender |
+----------+--------+
+----------+--------+
+----------+----------+--------+
|          | ageGroup | gender |
+----------+----------+--------+
| age25-34 |   male   |  100   |
+----------+----------+--------+
+----------

## Engagement and content sharing

In [4]:
import os
import googleapiclient.discovery
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow

## achieving video ids

# Set up OAuth credentials
os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"  # Only for development/testing
flow = InstalledAppFlow.from_client_secrets_file(
    "xav_client.json",
    scopes=["https://www.googleapis.com/auth/youtube.readonly"]
)
credentials = flow.run_local_server(port=0)

# Create a YouTube Data API client
youtube = googleapiclient.discovery.build("youtube", "v3", credentials=credentials)

# Get the playlist ID of the channel's uploaded videos
channel_id = "UCaufk4I0YqJZ-yIk8TNMS9w"
response = youtube.channels().list(
    part="contentDetails",
    id=channel_id
).execute()

if "items" in response:
    playlist_id = response["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]

    # Retrieve the list of videos in the playlist
    videos = []
    next_page_token = None
    while True:
        playlist_items_response = youtube.playlistItems().list(
            part="contentDetails",
            playlistId=playlist_id,
            maxResults=50,
            pageToken=next_page_token
        ).execute()

        videos.extend(playlist_items_response["items"])
        next_page_token = playlist_items_response.get("nextPageToken")

        if not next_page_token:
            break

video_ids = []
for i in videos:
    video_id = i["contentDetails"]["videoId"]
    video_ids.append(video_id)
    



from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import csv

from tabulate import tabulate

SCOPES = ['https://www.googleapis.com/auth/yt-analytics.readonly']

API_SERVICE_NAME = 'youtubeAnalytics'
API_VERSION = 'v2'
CLIENT_SECRETS_FILE = 'xav_client.json'

def get_service():
  flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
  credentials = flow.run_local_server()
  # alternatively (https://github.com/onlyphantom/youtube_api_python/pull/3/files):
  # credentials = flow.run_local_server()
  return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

def execute_api_request(client_library_function, **kwargs):
  response = client_library_function(
    **kwargs
  ).execute()
  return response

def create_table(table, headers=None):
    if headers:
        headerstring = "\t{}\t" * len(headers)
        print(headerstring.format(*headers))

    rowstring = "\t{}\t" * len(table[0])

    for row in table:
        print(rowstring.format(*row))

def create_csv(table, headers=None, filename=f'output{video_id}.csv'):
    with open(filename, 'w') as f:
        writer = csv.writer(f)
        if headers:
            writer.writerow(headers)
        writer.writerows(table)

if __name__ == '__main__':
    youtubeAnalytics = get_service()
    for i in video_ids:
        result = execute_api_request(
          youtubeAnalytics.reports().query,
          ids='channel==UCaufk4I0YqJZ-yIk8TNMS9w',
          startDate='2019-10-06',
          endDate='2023-10-31',
          metrics= 'shares',
          dimensions= 'sharingService',
          filters= f"video=={i}"
          
      )
        headers = ['sharing service']
        # create_table(result['rows'], headers=headers)
        create_csv(result['rows'], headers=headers,filename= f'{i}.csv' )
        print(tabulate(result['rows'], headers=headers, tablefmt="pretty"))

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=700561766577-d307303isgapjpl69e7lgtv2gitc665l.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A60107%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=JZWruhbv4dzf7j34TILtppOlVEMhWU&access_type=offline
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=700561766577-d307303isgapjpl69e7lgtv2gitc665l.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyt-analytics.readonly&state=fMFZngF1uUf8ZM43vz7wTTWrAHD5Pa&access_type=offline
+-----------------+
| sharing service |
+-----------------+
+-----------------+
+-------------------------------+-----------------+
|                               | sharing service |
+-------------------------------+-----------------+
|           FACEBOOK        

## Audience retention

In [6]:
import os
import googleapiclient.discovery
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow

## achieving video ids

# Set up OAuth credentials
os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"  # Only for development/testing
flow = InstalledAppFlow.from_client_secrets_file(
    "xav_client.json",
    scopes=["https://www.googleapis.com/auth/youtube.readonly"]
)
credentials = flow.run_local_server(port=0)

# Create a YouTube Data API client
youtube = googleapiclient.discovery.build("youtube", "v3", credentials=credentials)

# Get the playlist ID of the channel's uploaded videos
channel_id = "UCaufk4I0YqJZ-yIk8TNMS9w"
response = youtube.channels().list(
    part="contentDetails",
    id=channel_id
).execute()

if "items" in response:
    playlist_id = response["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]

    # Retrieve the list of videos in the playlist
    videos = []
    next_page_token = None
    while True:
        playlist_items_response = youtube.playlistItems().list(
            part="contentDetails",
            playlistId=playlist_id,
            maxResults=50,
            pageToken=next_page_token
        ).execute()

        videos.extend(playlist_items_response["items"])
        next_page_token = playlist_items_response.get("nextPageToken")

        if not next_page_token:
            break

video_ids = []
for i in videos:
    video_id = i["contentDetails"]["videoId"]
    video_ids.append(video_id)
    



from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import csv

from tabulate import tabulate

SCOPES = ['https://www.googleapis.com/auth/yt-analytics.readonly']

API_SERVICE_NAME = 'youtubeAnalytics'
API_VERSION = 'v2'
CLIENT_SECRETS_FILE = 'xav_client.json'

def get_service():
  flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
  credentials = flow.run_local_server()
  # alternatively (https://github.com/onlyphantom/youtube_api_python/pull/3/files):
  # credentials = flow.run_local_server()
  return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

def execute_api_request(client_library_function, **kwargs):
  response = client_library_function(
    **kwargs
  ).execute()
  return response

def create_table(table, headers=None):
    if headers:
        headerstring = "\t{}\t" * len(headers)
        print(headerstring.format(*headers))

    rowstring = "\t{}\t" * len(table[0])

    for row in table:
        print(rowstring.format(*row))

def create_csv(table, headers=None, filename=f'output{video_id}.csv'):
    with open(filename, 'w') as f:
        writer = csv.writer(f)
        if headers:
            writer.writerow(headers)
        writer.writerows(table)

if __name__ == '__main__':
    youtubeAnalytics = get_service()
    for i in video_ids:
        result = execute_api_request(
          youtubeAnalytics.reports().query,
          ids='channel==UCaufk4I0YqJZ-yIk8TNMS9w',
          startDate='2019-10-06',
          endDate='2023-10-31',
          metrics= 'audienceWatchRatio,relativeRetentionPerformance',
          dimensions= 'elapsedVideoTimeRatio',
          filters= f"video=={i}"
          
      )
        headers = ['audienceWatchRatio','relativeRetentionPerformance']
        # create_table(result['rows'], headers=headers)
        create_csv(result['rows'], headers=headers,filename= f'{i}.csv' )
        print(tabulate(result['rows'], headers=headers, tablefmt="pretty"))

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=700561766577-d307303isgapjpl69e7lgtv2gitc665l.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A60227%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=NPKkZZlNJ8nJ4YG97ODXqfQhcd79pt&access_type=offline
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=700561766577-d307303isgapjpl69e7lgtv2gitc665l.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyt-analytics.readonly&state=7nJhNBxEEb93RceHSKsngz26CsT2Om&access_type=offline
+--------------------+------------------------------+
| audienceWatchRatio | relativeRetentionPerformance |
+--------------------+------------------------------+
+--------------------+------------------------------+
+------+---------------------+------------------